In [ ]:
import copy

In [ ]:
'''
## Objects
==========
Node = {
    'node_id'            : _val_,
    'num_ports'.         : _val_,
    'queue_arrival_rate' : [<input_port>][<output_port>],   # λ_(i→j)
    'queue_service_rate' : [<input_port>][<output_port>],   # μ_(i→j)
    'queue_occupancy'    : [<input_port>][<output_port>],   # q_(i→j) 
    <port_id> : {
        'agg_input_arrival_rate'  : _val_,   # λ_i
        'agg_input_service_rate'  : _val_,   # μ_i
        'agg_output_arrival_rate' : _val_,   # λ_j
        'agg_output_service_rate' : _val_,   # μ_i

        'queue_capacity'          : _val_,   # Q_i

        <output_port_id>          : { 'flow_ids': [...]},      # list of flows this path through the router
        }
    }

Routing Table = {
    'current_node_id' : {
        'destination_node_id : <output_port>,
        }
    }

Flow Table = {
    <flow_id> : {
        'rate' : _val_,




# Usage examples
================
> Sets the arrival rate for traffic entering input port i=1 and leaving output port j=2
Node['queue_arrival_rate'][1][2] = 10

> Sets the aggregate arrival rate of traffic leaving output port j=2
Node[2]['agg_output_arrival_rate'] = 30

'''


In [ ]:
def SteadyRouter(router):
    # LOOP OVER ALL INPUT AND OUTPUT PORTS
    
    # Calculate λj for each output port
    for j in range(router['num_ports']):
        router[j]['agg_output_arrival_rate'] = 0
        for i in range(num_ports):
            router[j]['agg_output_arrival_rate'] += router['queue_arrival_rate'][i][j]
    
    # Calculate μi→j proportionally
    for i in range(router['num_ports']):
        for j in range(router['num_ports']):
            if router[j]['agg_output_arrival_rate'] > 0:
                router['queue_service_rate'][i][j] = \
                    (router['queue_arrival_rate'][i][j] / router[j]['agg_output_arrival_rate']) *\
                        router[j]['agg_output_service_rate']
            else:
                router['queue_service_rate'][i][j] = 0

    # Update queue occupancies; balance arrivals and service rates
    for i in range(router['num_ports']):
        qx = router[i]['queue_capacity'] # Remaining space in queue to be shared proportionally
        td = 0                           # Total rate differentials for proportional sharing

        for j in range(router['num_ports']):
            # Group 1: same arrival and service rate
            if router['queue_arrival_rate'][i][j] == router['queue_service_rate'][i][j]:
                qx = qx - router['queue_occupancy'][i][j]

            # Group 2: arrival rate less than service rate
            elif router['queue_arrival_rate'][i][j] < router['queue_service_rate'][i][j]:
                router['queue_occupancy'][i][j] = 0
                router['queue_service_rate'][i][j] = router['queue_arrival_rate'][i][j]

            # Group 3: arrival rate more than service rate (now only accumulate rate differentials)
            else:
                td = td + (router['queue_arrival_rate'][i][j] - router['queue_service_rate'][i][j])

        for j in range(router['num_ports']):
            if router['queue_arrival_rate'][i][j] > router['queue_service_rate'][i][j]:
                my_td = router['queue_arrival_rate'][i][j] - router['queue_service_rate'][i][j]
                router['queue_occupancy'][i][j] = (my_td/td) * qx
                router['queue_arrival_rate'][i][j] = router['queue_service_rate'][i][j]


        # Group 3: arrival rate more than service rate

In [ ]:
# Update the arrival rate for all cross-flows through the router
def UpdateArrival(router):
    for i in range(num_ports):
        for j in range(num_ports):
            router['queue_arrival_rate'][i][j] = 0
            for flow_id in router[i][j]['flow_ids']:
                router['queue_arrival_rate'][i][j] += flow_table[flow_id]['rate']

        router[i]['agg_input_arrival_rate'] = 0
        for j in range(num_ports):
            router[i]['agg_input_arrival_rate'] += router['queue_arrival_rate'][i][j]

In [ ]:
def SteadyNetwork():
    routers_to_process = [] # list of routers to be steadied
    
    for node in end_nodes():
        UpdateArrival(node)

        # Initialize input queuing occupancy
        for i in range(node['num_ports']):
            for j in range(node['num_ports']):
                if node[i]['agg_input_arrival_rate'] > 0:
                    node['queue_occupancy'][i][j] = \
                        (node['queue_arrival_rate'][i][j] / node[i]['agg_input_arrival_rate']) * \
                            node['queue_occupancy'][i][j]
                else:
                    node['queue_occupancy'][i][j] = 0

        for j in range(node['num_ports']):
            output_port_arrival_rate = 0
            for i in range(node['num_ports']):
                output_port_arrival_rate += node['queue_arrival_rate'][i][j]
                node[j]['agg_output_service_rate'] = min(output_port_arrival_rate, link_transmission_rate)

            if node[j]['agg_output_service_rate'] > 0:
                # Add the router if there’s flow through it
                routers_to_process.append(copy.deepcopy(node))

In [ ]:
### This section contains the input to the model and variables that must be initilized
link_transmission_rate = 5

# Initialize table of flows and their properties 
flow_table = {0: {'rate': 10, 'source': 0, 'destination': 4} }

# Initialize all end nodes
end_nodes = []

# Initialize all router nodes
routers = []

# Static routing table for finding path through the network
routing_table = {0:     # nodeid
                 {1: 1, # dst : output_port
                  2: 2,
                  3: 3},
                1:
                 {},
                }


In [ ]:
### MAIN


for flow_id in flow_table.keys():
    flow_rate = flow_table[flow_id]['rate']
    src_node_id = flow_table[flow_id]['source']
    dst_node_id = flow_table[flow_id]['destination']

    # Add flow to source endnode and balance the network

